# Graph Neural Networks (GNN): Advanced Tutorial

**Graph Neural Networks (GNNs)** are a class of neural networks that operate on graph-structured data.
They are used in social networks, molecules, recommendation systems, and more.

In this tutorial, we build a GCN (Graph Convolutional Network) using PyTorch Geometric on a node classification task.

## 1. Install Required Libraries

In [ ]:
# !pip install torch torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html


## 2. Import Libraries

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt


## 3. Load Dataset (Cora)

In [ ]:
dataset = Planetoid(root='data/', name='Cora')
data = dataset[0]

print(data)
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of node features: {dataset.num_node_features}')


## 4. Define GCN Model

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


## 5. Train the Model

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')


## 6. Evaluate the Model

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')


## 7. Summary

- GNNs learn representations of nodes using neighborhood aggregation
- GCN is a basic yet powerful model for node classification
- PyTorch Geometric simplifies working with graph datasets
- Try other models: GAT, GraphSAGE, GIN